# Автоматизация ежедневной отчетности по приложению

## Описание проекта

Необходимо автоматизировать отправку ежедневного отчета по работе всего приложения с помощью Airflow. В отчете должна быть информация и по ленте новостей, и по сервису сообщений. Отчет должен содержать ключевые метрики, позволяющие оценить работу приложения, так же отображать их динамику. К отчету необходимо приложить графики или файлы, помогающие сделать отчет более наглядным и информативным.

## Данные

Таблица feed_actions - данные по ленте новостей:
 - user_id - уникальный номер пользователя
 - post_id - уникальный номер поста
 - action - лайк или просмотр
 - time - время события
 - gender - пол пользователя
 - age - возраст пользователя
 - country - страна пользователя
 - city - город пользователя
 - os - операционная система устройства пользователя
 - sourse - источник привлечения пользователя (organic - поисковик, ads - реклама)
 - exp_group - номер экспериментальной группы
 
Таблица message_actions - данные по сервису обмена сообщениями:
 - user_id - уникальный номер пользователя
 - reciever_id - уникальный номер получетеля
 - time - время события
 - source - источник привлечения пользователя (organic - поисковик, ads - реклама)
 - exp_group - номер экспериментальной группы
 - gender - пол пользователя
 - age - возраст пользователя
 - country - страна пользователя
 - city - город пользователя
 - os - операционная система устройства пользователя

## Результат

Ежедневный отчет отправлется в телеграм в 11 утра.  
Отчет содержит текстовое сообщение со значениями метрик за вчера, позавчера и семь дней назад и три набора графиков с динамикой метрик за последние три недели.
В отчете отображаются метрики:
 - Лента новостей:
   - DAU
   - CTR
   - просмотры
   - лайки
 - Сервис сообщений:
   - DAU
   - отправлено сообщений
 - Пользовательская активность:
   - новые пользователи в ленте
   - просмотры на пользователя
   - лайки на пользователя
   - просмотры на пост
   - лайки на пост
   - отправлено сообщений на пользователя
   

 

## Скрипт сборки отчета

In [ ]:
# coding=utf-8

import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse as ph
from datetime import datetime, timedelta

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220820'
}

default_args = {
    'owner': 'e-varshavskaja-10',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 8, 25), 
}

schedule_interval = '0 11 * * *'
my_token = '**********:_______________________________'
bot = telegram.Bot(token=my_token)
chat_id = -*********

yesterday = (datetime.now() - timedelta(days=1)).date()
two_days = (datetime.now() - timedelta(days=2)).date()
week_ago = (datetime.now() - timedelta(days=7)).date()

def global_report():
        

    q1 = ''' SELECT 
                toDate(time) as date,
                COUNT(DISTINCT user_id) as users,
                SUM(action='view') as views,
                SUM(action='like') as likes,
                round((likes/views * 100), 1) as ctr,
                round(SUM(action='view')/COUNT(distinct user_id), 1) as views_per_user,
                round(SUM(action='like')/COUNT(distinct user_id), 1) as likes_per_user,
                round(SUM(action='view')/COUNT(distinct post_id), 1) as views_per_post,
                round(SUM(action='like')/COUNT(distinct post_id), 1) as likes_per_post
             FROM simulator_20220820.feed_actions
             WHERE toDate(time) between yesterday() - 20 and yesterday()
             GROUP BY toDate(time)
             ORDER BY date
             
             '''
    

    q2 = ''' SELECT 
                toDate(time) as date,
                COUNT(distinct user_id) as users,
                COUNT(user_id) as sent,
                round(COUNT(user_id)/COUNT(distinct user_id), 1) as msg_per_user
             FROM simulator_20220820.message_actions
             WHERE toDate(time) between yesterday() - 20 and yesterday()
             GROUP BY toDate(time)
             ORDER BY date

         '''    

    q3 = ''' SELECT 
                date,
                COUNT(users) users
             FROM (SELECT 
                      DISTINCT user_id users,
                      MIN(toDate(time)) as date
                   FROM simulator_20220820.feed_actions
                   GROUP BY user_id)
             WHERE date between yesterday() - 20 and yesterday()
             GROUP BY date

         '''
    feed = ph.read_clickhouse(query=q1, connection=connection)
    messages = ph.read_clickhouse(query=q2, connection=connection)
    new_feed = ph.read_clickhouse(query=q3, connection=connection)
        
          
 
    def transform_for_one(data, metric):
        one = data.query('date == @yesterday').reset_index()[metric][0]
        return one

    def transform_for_two(data, metric):
        two = data.query('date == @two_days').reset_index()[metric][0]
        return two

    def transform_for_week(data, metric):
        week = data.query('date == @week_ago').reset_index()[metric][0]
        return week
    
    one_dau = transform_for_one(feed, 'users')
    one_views = transform_for_one(feed, 'views')
    one_likes = transform_for_one(feed, 'likes')
    one_ctr = transform_for_one(feed, 'ctr')
    one_dau_msg = transform_for_one(messages, 'users')
    one_msg_sent = transform_for_one(messages, 'sent')
    one_new = transform_for_one(new_feed, 'users')
    one_avg_view = transform_for_one(feed, 'views_per_user')
    one_avg_like = transform_for_one(feed, 'likes_per_user')
    one_post_view = transform_for_one(feed, 'views_per_post')
    one_post_like = transform_for_one(feed, 'likes_per_post')
    one_avg_msg = transform_for_one(messages, 'msg_per_user')
    
    two_dau = transform_for_two(feed, 'users')
    two_views = transform_for_two(feed, 'views')
    two_likes = transform_for_two(feed, 'likes')
    two_ctr = transform_for_two(feed, 'ctr')
    two_dau_msg = transform_for_two(messages, 'users')
    two_msg_sent = transform_for_two(messages, 'sent')
    two_new = transform_for_two(new_feed, 'users')
    two_avg_view = transform_for_two(feed, 'views_per_user')
    two_avg_like = transform_for_two(feed, 'likes_per_user')
    two_post_view = transform_for_two(feed, 'views_per_post')
    two_post_like = transform_for_two(feed, 'likes_per_post')
    two_avg_msg = transform_for_two(messages, 'msg_per_user')
    
    week_dau = transform_for_week(feed, 'users')
    week_views = transform_for_week(feed, 'views')
    week_likes = transform_for_week(feed, 'likes')
    week_ctr = transform_for_week(feed, 'ctr')
    week_dau_msg = transform_for_week(messages, 'users')
    week_msg_sent = transform_for_week(messages, 'sent')
    week_new = transform_for_week(new_feed, 'users')
    week_avg_view = transform_for_week(feed, 'views_per_user')
    week_avg_like = transform_for_week(feed, 'likes_per_user')
    week_post_view = transform_for_week(feed, 'views_per_post')
    week_post_like = transform_for_week(feed, 'likes_per_post')
    week_avg_msg = transform_for_week(messages, 'msg_per_user')


                    
    msg = f''' ЕЖЕДНЕВНЫЙ ОТЧЕТ ПО ПРИЛОЖЕНИЮ 
Лента новостей(вчера/позавчера/неделю назад):\n 
DAU - {one_dau}/{two_dau}/{week_dau},
CTR - {one_ctr}%/{two_ctr}%/{week_ctr}%,
Просмотры - {one_views}/{two_views}/{week_views},
Лайки - {one_likes}/{two_likes}/{week_likes}\n\n
Сервис сообщений (вчера/позавчера/неделю назад):\n
DAU - {one_dau_msg}/{two_dau_msg}/{week_dau_msg},
Отправлено сообщений - {one_msg_sent}/{two_msg_sent}/{week_msg_sent}\n\n
Пользовательская активность(вчера/позавчера/неделю назад):\n
Новые пользователи в ленте - {one_new}/{two_new}/{week_new}
Просмотры на пользователя - {one_avg_view}/{two_avg_view}/{week_avg_view}
Лайки на пользователя - {one_avg_like}/{two_avg_like}/{week_avg_like}
Просмотры на пост - {one_post_view}/{two_post_view}/{week_post_view}
Лайки на пост - {one_post_like}/{two_post_like}/{week_post_like}
Отправлено сообщений на пользователя - {one_avg_msg}/{two_avg_msg}/{week_avg_msg}
           
             '''
    bot.sendMessage(chat_id=chat_id, text=msg)       
        
    fig, axes = plt.subplots(2, 2, figsize=(15,10))
    fig.suptitle('Лента новостей', size=15)
    sns.lineplot(x=feed['date'], y=feed['users'], ax=axes[0, 0])
    sns.lineplot(x=feed['date'], y=feed['ctr'], ax=axes[0, 1])
    sns.lineplot(x=feed['date'], y=feed['views'], ax=axes[1,0])
    sns.lineplot(x=feed['date'], y=feed['likes'], ax=axes[1,1])
    axes[0,0].set_title('DAU')
    axes[0,0].set_ylabel('Пользователи')
    axes[0,0].set_xlabel('')
    axes[0,1].set_title('CTR')
    axes[0,1].set_ylabel('%')
    axes[0,1].set_xlabel('')
    axes[1,0].set_title('Просмотры')
    axes[1,0].set_ylabel('Пользователи')
    axes[1,0].set_xlabel('')
    axes[1,1].set_title('Лайки')
    axes[1,1].set_ylabel('Пользователи')
    axes[1,1].set_xlabel('')
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = 'actions.png'
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)
    
    
    fig, axes = plt.subplots(1, 2, figsize=(17,5))
    fig.suptitle('Сервис сообщений', size=15)
    sns.lineplot(x=messages['date'], y=messages['users'], ax=axes[0])
    sns.lineplot(x=messages['date'], y=messages['sent'], ax=axes[1])
    axes[0].set_title('DAU')
    axes[0].set_ylabel('Пользователи')
    axes[0].set_xlabel('')
    axes[1].set_title('Отправленные сообщения')
    axes[1].set_ylabel('Сообщения')
    axes[1].set_xlabel('')
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = 'actions.png'
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)
        
        
    fig, axes = plt.subplots(3, 2, figsize=(15,15))
    fig.suptitle('Пользовательская активность', size=15)
    sns.lineplot(x=new_feed['date'], y=new_feed['users'], ax=axes[0, 0])
    sns.lineplot(x=messages['date'], y=messages['msg_per_user'], ax=axes[0, 1])
    sns.lineplot(x=feed['date'], y=feed['views_per_user'], ax=axes[1,0])
    sns.lineplot(x=feed['date'], y=feed['likes_per_user'], ax=axes[1,1])
    sns.lineplot(x=feed['date'], y=feed['likes_per_post'], ax=axes[2,0])
    sns.lineplot(x=feed['date'], y=feed['views_per_post'], ax=axes[2,1])
    axes[0,0].set_title('Новые пользователи')
    axes[0,0].set_ylabel('Пользователи')
    axes[0,0].set_xlabel('')
    axes[0,1].set_title('Количество сообщений на пользователя')
    axes[0,1].set_ylabel('Сообщения')
    axes[0,1].set_xlabel('')
    axes[1,0].set_title('Количество просмотров на пользователя')
    axes[1,0].set_ylabel('Просмотры')
    axes[1,0].set_xlabel('')
    axes[1,1].set_title('Количество лайков на пользователя')
    axes[1,1].set_ylabel('Лайки')
    axes[1,1].set_xlabel('')
    axes[2,0].set_title('Количество просмотров на пост')
    axes[2,0].set_ylabel('Просмотры')
    axes[2,0].set_xlabel('')
    axes[2,1].set_title('Количество лайков на пост')
    axes[2,1].set_ylabel('Лайки')
    axes[2,1].set_xlabel('')
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = 'actions.png'
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)
 

    
@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def dag_evarshavskaya_dailybot():
    
    @task
    def final_report():
        
        global_report()
    
    final_report()
    
dag_evarshavskaya_dailybot = dag_evarshavskaya_dailybot()